In [1]:
# Makes print and division act like Python 3
from __future__ import print_function, division

# Import the usual libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Enable inline plotting at lower left
%matplotlib inline

from IPython.display import display, Latex, clear_output
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
import pynrc
from pynrc import nrc_utils
from pynrc.nrc_utils import S
from pynrc.obs_nircam import model_to_hdulist
pynrc.setup_logging('WARNING', verbose=False)

In [6]:
#                source,    dist,  fov, age,  sptype, kmag
args_sources = [('HD10647',  17.43, 14, 1000, 'F9V', 4.3),
                ('HD107146', 27.50, 14,  150, 'G2V', 5.5),
                ('HD181327', 51.80,  7,   12, 'F6V', 5.9),
                ('HD61005',  34.50,  7,  100, 'G8V', 6.5),
                ('HD32297',  113.0,  7,   30, 'A0V', 7.6)]

ref_sources  = [('iotHor',   'F8V',  4.1),
                ('HD111398', 'G5V',  5.5),
                ('HR7297',   'F7V',  5.1),
                ('HD56161',  'G5IV', 4.9),
                ('HD31411',  'A0V',  6.4)]

In [7]:
bp_k = pynrc.bp_2mass('k')

## ND Target Acquisition Times

In [19]:
nrc = pynrc.NIRCam(filter='F335M', pupil='CIRCLYOT', ND_acq=True,
                   wind_mode='WINDOW', xpix=64, ypix=64)

for i in range(len(args_sources)):

    name_sci, dist_sci, fov, age, spt_sci, kmag_sci = args_sources[i]
    name_ref, spt_ref, kmag_ref = ref_sources[i]

    # Science target and reference star spectra
    sp_sci = pynrc.stellar_spectrum(spt_sci, kmag_sci, 'vegamag', bp_k, catname='ck04models')
    sp_ref = pynrc.stellar_spectrum(spt_ref, kmag_ref, 'vegamag', bp_k, catname='ck04models')
    sp_sci.name = name_sci
    sp_ref.name = name_ref
    
    
    print(name_sci)

    ng_levels = [65]
    for sp in [sp_sci, sp_ref]:
        for ng in ng_levels: 
            pynrc.setup_logging('ERROR', verbose=False)
            tbl = nrc.ramp_optimize(sp, snr_goal=50, snr_frac=0.01, well_frac_max=0.5, verbose=False,
                                    ng_min=ng, ng_max=ng, nint_max=1, patterns=['RAPID'])
            nrow = len(tbl)
            if nrow>0:
                print(tbl)
                break

    print('')

pynrc.setup_logging('WARNING', verbose=False)

HD10647
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
RAPID        65    1      3.26      3.26      3.32    152.0    0.055   83.472
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
RAPID        65    1      3.26      3.26      3.32    167.1    0.067   91.749

HD107146
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
RAPID        65    1      3.26      3.26      3.32     84.5    0.018   46.393
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
RAPID        65    1      3.26      3.26      3.32     84.5    0.018   46.393

HD181327
 Pattern   NGRP NINT   t_int     t_e

In [23]:
# HD 32297 and reference star are too faint
# Turn off ND_acq for faint TA position
nrc = pynrc.NIRCam(filter='F335M', pupil='CIRCLYOT', ND_acq=False,
                   wind_mode='WINDOW', xpix=64, ypix=64)

i=4
name_sci, dist_sci, fov, age, spt_sci, kmag_sci = args_sources[i]
name_ref, spt_ref, kmag_ref = ref_sources[i]

# Science target and reference star spectra
sp_sci = pynrc.stellar_spectrum(spt_sci, kmag_sci, 'vegamag', bp_k)
sp_ref = pynrc.stellar_spectrum(spt_ref, kmag_ref, 'vegamag', bp_k)
sp_sci.name = name_sci
sp_ref.name = name_ref


print(name_sci)

ng_levels = [5,9,17,33,65]
for sp in [sp_sci, sp_ref]:
    for ng in ng_levels: 
        pynrc.setup_logging('WARNING', verbose=False)
        tbl = nrc.ramp_optimize(sp, snr_goal=100, snr_frac=0.01, well_frac_max=0.5, verbose=False,
                                ng_min=ng, ng_max=ng, nint_max=1, patterns=['RAPID'])
        nrow = len(tbl)
        if nrow>0:
            print(tbl)
            break


HD32297
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
RAPID         5    1      0.25      0.25      0.31    185.6    0.112  335.849
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
RAPID         5    1      0.25      0.25      0.31    332.1    0.338  600.879


## Astrometric Confirmation Settings

Want to be sensitive to objects with brightnesses of 10-18 mags at 3.3um

In [24]:
nrc = pynrc.NIRCam(filter='F335M', pupil='CIRCLYOT', wind_mode='FULL')

In [33]:
sp_dim = pynrc.stellar_spectrum('flat', 16, 'vegamag', nrc.bandpass)
sp_bright = pynrc.stellar_spectrum('flat', 11, 'vegamag', nrc.bandpass)
tbl = nrc.ramp_optimize(sp_dim, sp_bright, snr_goal=30, snr_frac=0.1, ng_min=3, ng_max=10, nint_min=3,
                        well_frac_max=1.2, verbose=True)


BRIGHT1
BRIGHT2
DEEP2
DEEP8
MEDIUM2
MEDIUM8
RAPID
SHALLOW2
SHALLOW4
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
BRIGHT1       3    3     53.68    161.05    193.27     64.8    1.043    4.657
RAPID         3    3     32.21     96.63    128.85     36.2    0.626    3.185
